<a href="https://colab.research.google.com/github/mch60131242/Home/blob/main/Bert_IMBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [7]:
, load_metric
from transformers import BertTokenizer,BertForSequenceClassification

## 버트모델 불러오기

from transformers import TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset, load_metric

In [8]:
## IMDB 영화리뷰 데이터,# 자연어 처리의 대표적인 dataset
data = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
import pandas as pd
import numpy as np
df =pd.DataFrame(data['train'])

In [17]:
## 길이 확인
df['length'] =df['text'].apply(lambda x: len(x))

In [26]:
##tokenizer, model 불러오기

,text,label,length
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,1640
1,"""I Am Curious: Yellow"" is a risible and preten...",0,1294
2,If only to avoid making this type of film in t...,0,528
3,This film was probably inspired by Godard's Ma...,0,706
4,"Oh, brother...after hearing about this ridicul...",0,1814


In [21]:
model_name = 'bert-base-uncased'
model= BertForSequenceClassification.from_pretrained(model_name)

token =BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
args = TrainingArguments('test-trainer')

In [24]:
def tokenize_function(example):
  return token(example['text'], truncation=True) ## truncation <-데이터의 max_sequence가 넘는걸 미리 잘라서 넣어줘라  


In [25]:
token_datasets = data.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [28]:
token_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [27]:
data_collator = DataCollatorWithPadding(tokenizer=token)

In [31]:
from transformers import Trainer

trainer =Trainer(
    model,
    args,
    train_dataset = token_datasets['train'],
    eval_dataset =token_datasets['test'],
    data_collator = data_collator,
    tokenizer=token
)

In [ ]:
trainer.train()